## REGRESSION

**Question** Do you remember what the regression is?

*Nasza odpowiedź*
Regresja - metoda statystyczna pozwalająca na opisanie współzmienności kilku zmiennych przez dopasowanie do nich funkcji. Umożliwia przewidywanie nieznanych wartości jednych wielkości na podstawie znanych wartości innych.

Today's analysis we will run on dataset 'car-mpg' which has the following columns: cylinders, displacement, horsepower, weight, acceleration, model year, origin and mpg. Mpg is decision attribute and stands for miles per gallon, which basically means the reverse of combustion (pl. spalanie). The dataset comes from https://archive.ics.uci.edu/ml/datasets/auto+mpg However, it does not have the original data, but normalized version.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from scipy.stats import f, f_oneway
import statsmodels.api as sm
%matplotlib inline
plt.style.use('ggplot')

In [ ]:
data = pd.read_csv('car-mpg.csv', header=None)
data.head()

As the first step of the task let's get to know with our data and its distribution.

In [ ]:
columns = ['cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model_year', 'origin', 'mpg']
# done Iterate through the columns (attributes) in our dataset and draw their histograms.
for col in range(len(columns)):
    plt.figure(figsize=(6, 4))
    plt.hist(data[col], bins=20, color='blue', alpha=0.7)
    plt.title(f'{columns[col]}')
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.show()
# Which attributes can be potentially more interesting in further analysis? 
# todo
data.columns = columns #na potem, żeby było po nazwach a nie indeksach ;)

### SIMPLE LINEAR REGRESSION

**Task** Write your own function which calculates linear regression step by step and plot the results. <br/>
*Hint* If you do not remember how it was calculated, look here: https://en.wikipedia.org/wiki/Simple_linear_regression

In [ ]:
def LinearRegressionOne(x, y):
    #done write your linear regression function
    a = np.sum((x - np.mean(x)) * (y - np.mean(y))) / np.sum((x - np.mean(x)) ** 2)
    b = np.mean(y) - a * np.mean(x)
    return a,b

In [ ]:
a, b = LinearRegressionOne(data["horsepower"], data["mpg"]) # done run your function on attributes 'horsepower' and 'mpg'
Y =  b + a * data["horsepower"]
plt.plot(data["horsepower"], Y, label=f'Linear fit: y = {b:.2f} + {a:.2f}x')
plt.scatter(data["horsepower"], data["mpg"], color="blue")
plt.show()

**Task** As you already wrote your own function which calculates simple linear regression, you can now use implementation from sklearn package. This time split your data to train and test set (use first 40 observations as test ones) and plot your results. Compare the results from your own implementation and this one from sklearn. Did you get similar coefficients for your linear function?

*similar? - The same. Well, 0.000000000000004 off, so... **the same***

In [ ]:
from sklearn.linear_model import LinearRegression

# TODO simple linear regression with the usage of sklearn
def LinearRegressionOneScikit(x_train, y_train):
    x_train_reshaped = x_train.values.reshape(-1, 1)
    y_train_reshaped = y_train.values.reshape(-1, 1)
    model = LinearRegression()
    model.fit(x_train_reshaped, y_train_reshaped)
    a = model.coef_[0][0]
    b = model.intercept_[0]
    return a, b

X_train, X_test = data["horsepower"][:40], data["horsepower"][40:]
Y_train, Y_test = data["mpg"][:40], data["mpg"][40:]
a_custom, b_custom = LinearRegressionOne(X_train, Y_train)
a_sklearn, b_sklearn = LinearRegressionOneScikit(X_train, Y_train)

Y_custom =  b_custom + a_custom * data["horsepower"]
Y_sklearn = b_sklearn + a_sklearn * data["horsepower"]

plt.figure(figsize=(10, 6))
plt.scatter(X_test, Y_test, color="lightblue", label="Test Data")
plt.scatter(X_train, Y_train, color="aqua", label="Train Data")
plt.plot(data["horsepower"], Y_custom, label=f'Linear fit: y = {b:.2f} + {a:.2f}x', color="pink")
plt.plot(data["horsepower"], Y_sklearn, label=f'Linear fit: y = {b:.2f} + {a:.2f}x', color="red")
plt.legend()
plt.show()

print(f"Custom Implementation Coefficients: \n  Slope (a): {a_custom}, Intercept (b): {b_custom}")
print(f"Sklearn Implementation Coefficients: \n  Slope (a): {a_sklearn}, Intercept (b): {b_sklearn}")

#### Polynomial regression

We do not have to limit to only simple line but we also can create some other shapes. Run the following codes:

In [ ]:
pd.options.mode.chained_assignment = None  # default='warn'
x = data.iloc[:,2:3]
y = data.iloc[:,-1]

x[10] = x[2]**2  # 'x^2'

reg = linear_model.LinearRegression().fit(x.to_numpy(),y)
score = reg.score(x,y)

x_line = np.array(np.linspace(0,1,100))
y_line = reg.coef_[1]*x_line**2+ reg.coef_[0]*x_line+ reg.intercept_

plt.scatter(x[2],y)
plt.plot(x_line,y_line)

In [ ]:
x = data.iloc[:,4:5]
y = data.iloc[:,-1]

x[10]=x[4]**2  # 'x^2'
x[11]=x[4]**3  # 'x^3'

reg = linear_model.LinearRegression().fit(x,y)
score = reg.score(x,y)

x_line = np.array(np.linspace(0,1,100))
y_line =  reg.coef_[2]*x_line**3+ reg.coef_[1]*x_line**2+ reg.coef_[0]*x_line+ reg.intercept_

plt.scatter(x[4],y)
plt.plot(x_line,y_line)

### MULTIPLE LINEAR REGRESSION

**Question** Do you remember how the multiple linear regression works? <br/>

*Nasza odpowiedź*

Regresja wieloraka - zmienna objaśniana zależy od więcej niż jednej zmiennej, oblicza się macierzą

#### GRADIENT DECENT ALGORITHM
Gradient descent algorithm’s main objective is to minimise the cost function. It is one of the best optimisation algorithms to minimise errors (difference of actual value and predicted value). In a real world example, it is similar to find out a best direction to take a step downhill.

**Task** This time you are about to fill missing values in multiple linear regression implementation with the usage of batch gradient decent algorithm.

In [ ]:
x = data.iloc[:,:-1]
y = data.iloc[:,-1:]

# we add intercept to the vector of x to make it easier to process with other values
x['intercept'] = 1 

n_iterations = 1000 # TODO set the number of iterations
learning_rate = 0.1 # TODO set learning rate

weights = np.zeros((x.shape[1], 1))

print(x.shape[0])

for _ in range(n_iterations):
    y_pred = np.dot(x, weights)
    residuals = y_pred - y
    gradient_vector = np.dot(x.T, residuals)
    weights -= learning_rate/x.shape[0] * gradient_vector

# last weight is the value of intercept 
weights

**Task** You do not need to write your own implementation of multiple linear regression, you can also use sklearn implementation. This time your function should have params: independent variables *x* and dependent variable *y* and return (or just print) score and coefficients of the function (.coef_ and .intercept_).

In [ ]:
def MultipleLinearRegression(x_train, y_train):
    # DONE here goes your code
    model = LinearRegression()
    model.fit(x_train, y_train)
    return model.coef_, model.intercept_
    


coefficients, intercept, model = MultipleLinearRegression(x_train, y_train)

y_pred = model.predict(x_test)
r2 = r2_score(y_test, y_pred)

print("Coefficients:", coefficients)
print("Intercept:", intercept)
print(f"R^2 Score on Test Set: {r2:.2f}")



**Question** Compare the results from two above methods. Are they similar?

## SIGNIFICANCE REGRESSION COEFFICIENT

We can statistically check if the model (or specific parameter) is significant.

#### GLOBAL

**Task** Implement global test for checking significance. Fill the following methods according to your knowledge from the lecture.

In [ ]:
def getSSR(y,y_pred):
    #TODO
    return 

def getSSE(y,y_pred):
    #TODO
    return 

def getF(y, y_pred, n, m):
    #TODO
    return 

def getSignificanceRegressionCoefficient(x,y):
    #TODO
    return F,p

In [ ]:
xx = data.iloc[:,:-1].values
yy = data.iloc[:,-1:].values[:,0]

In [ ]:
F,p = getSignificanceRegressionCoefficient(xx,yy)
print(F)
if p < 0.05:
    print(" confirm " + str(p))
else:
    print(" reject " + str(p))

Let's check the correctness of your function.

In [ ]:
sm_stats = sm.OLS(yy, sm.add_constant(xx)).fit()
print(sm_stats.fvalue)

#### LOCAL

**Task** Now check the significance of each criterion. As the equation for global version is more general, we can use the function that was previously written.

In [ ]:
#TODO

## OUTLIERS

**Question** What is an outlier?

Let's look at the boxplot below. The points which are at the top and bottom of chart are outliers.

In [ ]:
fig, ax1 = plt.subplots(figsize=(3, 6))
plt.boxplot(data[4], 1)
plt.show()

The method that is used in this type of charts to determine outliers bases on quartiles. Firstly, we need to calculate IQR which stands for interquartile range and is the difference between the third and the first quartile. Usually stating the limit of whiskers is: [Q1-IQR * 1.5, Q3 + IQR * 1.5]. All values that are outside this range are treated as outliers. It is connected with normal distribution and distance greater than 3 std dev.

However, this method cannot cope correctly with multidimensional data. There can be outliers that are still close to median but overall are far from other points. 

We will use *statsmodel* package which calculates a few interesting measures.

In [ ]:
data.columns = ['cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model_year', 'origin', 'mpg']

model = sm.OLS(data.iloc[:,-1], data.iloc[:,:-1])
results = model.fit()
influence = results.get_influence()
sm_fr = influence.summary_frame()
sm_fr

**Task** Visualize residuals (*standard_resid*) with the usage of scatterplot. Draw also horizontal line on y-value = 0. Look at the chart. Are these values in a narrow row on y-value or you can identify that there are some outliers?

In [ ]:
#TODO

#### DFFITS

This measure calculates the influence of i-th example on our regression equation by omitting this observation and comparing this model to previous one. Boundary values for our influential observations are +/- 2 * sqrt(1/number of observations)

**Task** Draw a plot where you visualise values of dffits as vertical lines (see: plt.vlines()) and horizontal lines (plt.axhline()) equal to boundaries of influential observations.

In [ ]:
#TODO

**Task** Get indices of those observations that are influential for diffits.

In [ ]:
#TODO

#### COOK'S DISTANCE

Comparing to the previous, there is also made checking the influence of each observation. However, this time we check the difference of coefficients in our regression equation while we omit the i-th observation. The boundary of the influence is very often set to 4/number of observations.

**Task** Once again draw a plot where you visualise values of cooks_d as vertical lines and horizontal line equal to boundary of influential observations. *Tip* It might be useful to visualize square root of the values to make the chart more transparent.

In [ ]:
#TODO

**Task** Get indices of those observations that are influential for Cook's distance.

In [ ]:
#TODO

**Task** Check if there are elements that appear on both lists. Maybe one list is subset of the second?

In [ ]:
#TODO